In [1]:
import os
import warnings
warnings.filterwarnings(action='ignore')

import numpy as np
import pandas as pd

import torch
import torchaudio
import torchvision
import torch.nn as nn

from tqdm.notebook import tqdm
from torch.optim.lr_scheduler import ReduceLROnPlateau
from sklearn import metrics

from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

from matplotlib import pyplot as plt
from PIL import Image

import pickle


# Data preprocessing

In [2]:
TAGS = ['Blues', 'Jazz', 'Rock', 'R&B', 'Alternative Rock', 'Latin Music', 'Country',
 'Rap & Hip-Hop', 'Dance & Electronic', 'Reggae', 'Classical', 'Metal', 'Pop',
 'New Age', 'Folk', 'Gospel']

data = pd.read_csv("./MuMu_dataset/MuMu_dataset_all-label.csv", sep=",", index_col=0)


In [3]:
# Convert image files to tensor files & Save
trans = transforms.Compose([transforms.Resize((224, 224)),
                            transforms.ToTensor(),
                            transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

for idx in range(len(data)):
    img_path = data.iloc[idx].img_path
    img = Image.open(img_path).convert("RGB")
    torch.save(trans(img), img_path.replace(".jpg", ".pt"))

UnidentifiedImageError: cannot identify image file './MuMu_dataset/album_imgs/Rock/313eVoHHbwL._SL500_.gif'

In [3]:
class AlbumGenreDataset(Dataset):
    def __init__(self, data, group='debug'):
        """
        data (Pandas.DataFrame): ['amazon_id', 'title', 'artist', 'img_path', 'genre']
        group (String): split group e.g. train, valid
        """
        self.data = data
        self.group = group

    def __getitem__(self, index):
        item = self.data.iloc[index]
        img = torch.load(item.img_path.replace(".jpg", ".pt"))

        label = TAGS.index(item.genre)
        return img, label
        

    def __len__(self):
        return len(self.data)

In [4]:
data_num = len(data)
tr_data = AlbumGenreDataset(data[:int(data_num*0.6)], 'TRAIN')
va_data = AlbumGenreDataset(data[int(data_num*0.6):int(data_num*0.8)], 'VALID')
te_data = AlbumGenreDataset(data[int(data_num*0.8):], 'TEST')
print("(TRAIN, VALID, TEST) = ", len(tr_data), len(va_data), len(te_data))

(TRAIN, VALID, TEST) =  1346 449 449


In [5]:
BATCH_SIZE = 16
loader_train = DataLoader(tr_data, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
loader_valid = DataLoader(va_data, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)
loader_test = DataLoader(te_data, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

# Train simple baseline Model

In [6]:
class Baseline(nn.Module):
    def __init__(self, n_class=16):
        super(Baseline, self).__init__()
        self.conv0 = nn.Sequential(
            nn.Conv2d(3, out_channels=32, kernel_size=3, stride=1, padding=3),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=3))

        self.conv1 = nn.Sequential(
            nn.Conv2d(32, out_channels=32, kernel_size=3, stride=1, padding=3),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=3))

        self.conv2 = nn.Sequential(
            nn.Conv2d(32, out_channels=32, kernel_size=3, stride=1, padding=3),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=3, stride=3))
        
        self.final_pool = nn.AdaptiveAvgPool2d(1)
        self.linear = nn.Linear(32, n_class)
        
        
    def forward(self, x):
        x = self.conv0(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.final_pool(x)
        x = x.view(x.size(0), -1)
        out = self.linear(x)
        #This doesn't need because the CrossEntropyLoss criterion already includes a softmax function.
        #out = nn.Softmax()(x)
        return out

In [7]:
class Runner(object):
  def __init__(self, model, lr, weight_decay, sr, tags):
    self.optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=weight_decay)
    self.scheduler = ReduceLROnPlateau(self.optimizer, mode='min', factor=0.2, patience=5, verbose=True)
    self.learning_rate = lr
    self.stopping_rate = sr
    self.device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    self.model = model.to(self.device)
    self.criterion = torch.nn.CrossEntropyLoss().to(self.device)
    self.tags = tags

  def run(self, dataloader, epoch, mode='TRAIN'):
    self.model.train() if mode is 'TRAIN' else self.model.eval()

    epoch_loss = 0
    pbar = tqdm(dataloader, desc=f'{mode} Epoch {epoch:02}')  # progress bar
    for x, y in pbar:
      x = x.to(self.device)
      y = y.to(self.device)
      prediction = self.model(x)
      loss = self.criterion(prediction, y)
      if mode is 'TRAIN':
        loss.backward()
        self.optimizer.step()
        self.optimizer.zero_grad()

      batch_size = len(x)
      epoch_loss += batch_size * loss.item()
    epoch_loss = epoch_loss / len(dataloader.dataset)
    return epoch_loss

  def test(self, dataloader):
    self.model.eval()
    total = 0
    correct = 0

    pbar = tqdm(dataloader, desc=f'TEST')
    for x, y in pbar:
      x = x.to(self.device)
      y = y.to(self.device)
      output = self.model(x)

      _, predicted = torch.max(output.data, 1)
      total += y.size(0)
      correct += (predicted == y).sum().item()
      
    return 100 * correct / total

  def early_stop(self, loss, epoch):
    self.scheduler.step(loss, epoch)
    self.learning_rate = self.optimizer.param_groups[0]['lr']
    stop = self.learning_rate < self.stopping_rate
    return stop


In [13]:
# Training setup.
LR = 1e-3  # learning rate
SR = 1e-5  # stopping rate
NUM_EPOCHS = 50
WEIGHT_DECAY = 1e-5  # L2 regularization weight

model = Baseline()
runner = Runner(model=model, lr = LR, weight_decay = WEIGHT_DECAY, sr = SR, tags=TAGS)

In [14]:
min_valid_loss = 1000

for epoch in range(NUM_EPOCHS):
  # train & valid
  train_loss = runner.run(loader_train, epoch, 'TRAIN')
  valid_loss = runner.run(loader_valid, epoch, 'VALID')
  print("[Epoch %d/%d] [Train Loss: %.4f] [Valid Loss: %.4f]" %
        (epoch + 1, NUM_EPOCHS, train_loss, valid_loss))
  
  # save the model of minimum valid loss
  if valid_loss < min_valid_loss:
    min_valid_loss = valid_loss
    torch.save(model, './min_valid_model.pt')
  
  # early stopping
  if runner.early_stop(valid_loss, epoch + 1):
    break

print("min_valid_loss:", min_valid_loss)

TRAIN Epoch 00:   0%|          | 0/84 [00:00<?, ?it/s]

VALID Epoch 00:   0%|          | 0/28 [00:00<?, ?it/s]

[Epoch 1/50] [Train Loss: 2.4984] [Valid Loss: 2.3641]


TRAIN Epoch 01:   0%|          | 0/84 [00:00<?, ?it/s]

VALID Epoch 01:   0%|          | 0/28 [00:00<?, ?it/s]

...

VALID Epoch 42:   0%|          | 0/28 [00:00<?, ?it/s]

[Epoch 43/50] [Train Loss: 2.0563] [Valid Loss: 2.2425]


TRAIN Epoch 43:   0%|          | 0/84 [00:00<?, ?it/s]

VALID Epoch 43:   0%|          | 0/28 [00:00<?, ?it/s]

[Epoch 44/50] [Train Loss: 2.0610] [Valid Loss: 2.2429]
Epoch    44: reducing learning rate of group 0 to 8.0000e-06.
min_valid_loss: 2.2402010112668997


In [15]:
# load the min_valid_model & check accuracy
model = torch.load("./min_valid_model.pt")
runner = Runner(model=model, lr = LR, weight_decay = WEIGHT_DECAY, sr = SR, tags=TAGS)
aucs = runner.test(loader_test)
print(f'auc={aucs:.2f}%')

TEST:   0%|          | 0/28 [00:00<?, ?it/s]

auc=20.54%


# Train pre-trained model

In [15]:
class CNN_Classifier(nn.Module):
  def __init__(self, n_class=16):
    super(CNN_Classifier, self).__init__()
    self.pretrained_model = torchvision.models.resnet101(pretrained=True)
    self.linear = nn.Linear(1000, n_class)

  def forward(self, x):
    x = self.pretrained_model(x)
    x = x.view(x.size(0), -1)
    x = self.linear(x)
    out = nn.Softmax()(x)
    return out

In [16]:
# Training setup.
LR = 1e-3  # learning rate
SR = 1e-5  # stopping rate
NUM_EPOCHS = 30
WEIGHT_DECAY = 1e-5  # L2 regularization weight

model = CNN_Classifier()
runner = Runner(model=model, lr = LR, weight_decay = WEIGHT_DECAY, sr = SR, tags=TAGS)

In [17]:
for epoch in range(NUM_EPOCHS):
  train_loss = runner.run(loader_train, epoch, 'TRAIN')
  valid_loss = runner.run(loader_valid, epoch, 'VALID')
  print("[Epoch %d/%d] [Train Loss: %.4f] [Valid Loss: %.4f]" %
        (epoch + 1, NUM_EPOCHS, train_loss, valid_loss))
  if runner.early_stop(valid_loss, epoch + 1):
    break

TRAIN Epoch 00:   0%|          | 0/84 [00:00<?, ?it/s]

VALID Epoch 00:   0%|          | 0/28 [00:00<?, ?it/s]

[Epoch 1/30] [Train Loss: 2.7254] [Valid Loss: 2.6633]


TRAIN Epoch 01:   0%|          | 0/84 [00:00<?, ?it/s]

VALID Epoch 01:   0%|          | 0/28 [00:00<?, ?it/s]

...

VALID Epoch 17:   0%|          | 0/28 [00:00<?, ?it/s]

[Epoch 18/30] [Train Loss: 2.7240] [Valid Loss: 2.6655]


TRAIN Epoch 18:   0%|          | 0/84 [00:00<?, ?it/s]

VALID Epoch 18:   0%|          | 0/28 [00:00<?, ?it/s]

[Epoch 19/30] [Train Loss: 2.7240] [Valid Loss: 2.6633]
Epoch    19: reducing learning rate of group 0 to 8.0000e-06.


In [18]:
aucs = runner.test(loader_test)
print(f'auc={aucs:.2f}%')

TEST:   0%|          | 0/28 [00:00<?, ?it/s]

auc=20.09%
